In [ ]:
!pip install selenium

In [ ]:
import sys
sys.path.insert(0,'chromedriver')
from selenium import webdriver
from selenium.webdriver.common.by import By
from pandas import DataFrame
from datetime import datetime
from time import sleep
from re import sub

In [ ]:
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')

# Remove comment for headless execution
# chrome_options.add_argument('--headless')

In [ ]:
wd = webdriver.Chrome('chromedriver.exe', options=chrome_options)

In [ ]:
base_url = "https://www.bigsoccer.com/threads/going-to-qatar-2022-tickets-accommodations-etc.2118067/page-"

In [ ]:
wd.get(base_url + str(1))
pagenums = wd.find_elements(By.XPATH, "//div[contains(@class, 'PageNav')]/nav/a")
max_pagenum = int(pagenums[-2].text)
max_pagenum

742

In [ ]:
def remove_block_quotes():
    element = wd.find_elements(By.XPATH, "//div[contains(@class, 'bbCodeBlock')]")
    for e in element:
        try:
            wd.execute_script("""
                var element = arguments[0];
                element.parentNode.removeChild(element);
                """, e)
        except:
            pass

In [ ]:
for i in range(1, max_pagenum + 1):
    wd.get(base_url + str(1))
    remove_block_quotes()
    sections = wd.find_elements(By.XPATH, "//div[contains(@class, 'uix_message')]")

    all_comments = DataFrame({
        "user": [],
        "nationality": [],
        "current_location": [],
        "comment_date": [],
        "comment": []
    })

    for i, section in enumerate(sections):
        username = section.find_element(By.XPATH, "div/div/h3/div[contains(@class, 'uix_userTextInner')]/a[contains(@class, 'username')]")
        username = username.text

        nationality = None
        try:
            natnlty = section.find_element(By.XPATH, "div/div/div[contains(@class, 'extraUserInfo')]/dl/dd/span/span[contains(@class, 'bsNational__content')]")
            nationality = natnlty.text
        except:
            pass
        
        current_location = None
        try:
            location = section.find_element(By.XPATH, "div/div/div[contains(@class, 'extraUserInfo')]/dl/dd/a[contains(@itemprop, 'address')]")
            current_location = location.text
        except:
            pass
        
        date = section.find_element(By.XPATH, "div/div[contains(@class, 'messageDetails')]/span/a/span[contains(@class, 'DateTime')]")
        date = date.get_attribute("title")
        date = datetime.strptime(date, '%b %d, %Y at %I:%M %p')
        date = f"{date.year}-{date.month:02}-{date.day:02}"

        comment = section.find_element(By.XPATH, "div/div[contains(@class, 'messageContent')]/article")
        comment = comment.text

        all_comments.loc[i] = [ username, nationality, current_location, date, comment]
    # Check for 1 page by breaking
    # break
all_comments

,user,nationality,current_location,comment_date,comment
0,Danny Ascencio,Argentina,"Dallas, Texas",2021-10-22,"In the fifa website, says November 21- Decembe..."
1,mazinio,None,None,2021-10-23,Where can u aee that. Can u please share the l...
2,Paul Calixte,None,"Miami, FL",2021-10-23,"Hi everyone,\n\nWith the incoming proliferatio..."
3,Danny Ascencio,Argentina,"Dallas, Texas",2021-10-25,"I spoke too soon, it says it will be available..."
4,Danny Ascencio,Argentina,"Dallas, Texas",2021-10-25,https://www.fifa.com/tournaments/mens/worldcup...
5,Dr. Gamera,None,"Wheaton, Maryland",2021-10-25,The “November 21- December 17” range at that l...
6,Timanfaya,None,Southampton,2021-10-26,I expect tickets will still be on sale on thos...
7,mazinio,None,None,2021-10-28,https://www.reuters.com/lifestyle/s...anage-wo...
8,Timanfaya,None,Southampton,2021-10-28,It's nowhere near enough. Most of those hotel ...
9,mazinio,None,None,2021-10-28,I still hope that the 60k apartments and villa...


In [ ]:
all_comments.to_csv("comments.csv", index=False)